#### Import the standard libraries:
 - pandas
 - numpy
 - datetime

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

## Merging

Let's bring in some new data and start from scratch.


In [2]:
#comp1 = pd.read_excel('fina.xlsx', index=False, converters={'gvkey': '{:0>6}'.format})
comp1 = pd.read_excel('fina.xlsx', converters={'gvkey': '{:0>6}'.format})
comp1.head(20)

,gvkey,cik,datadate,tic,conm,ni,sale,cogs,oancf,at,lt,seq
0,331687,66740,2012-12-31,MMM,3M Company,4444.000,29904.000,15685.000,NaN,33876.000,15836.000,17575.000
1,331687,66740,2013-12-31,MMM,3M Company,4659.000,30871.000,16106.000,NaN,33550.000,15602.000,17502.000
2,331687,66740,2014-12-31,MMM,3M Company,4956.000,31821.000,16447.000,NaN,31269.000,18127.000,13109.000
3,331687,66740,2015-12-31,MMM,3M Company,4833.000,30274.000,15383.000,6420.000,32718.000,20971.000,11708.000
4,331687,66740,2016-12-31,MMM,3M Company,5050.000,30109.000,15040.000,6662.000,32906.000,22563.000,10298.000
5,331687,66740,2017-12-31,MMM,3M Company,4858.000,31657.000,16001.000,6240.000,37987.000,26365.000,11563.000
6,331687,66740,2018-12-31,MMM,3M Company,5349.000,32765.000,16682.000,6439.000,36500.000,26652.000,9796.000
7,331687,66740,2019-12-31,MMM,3M Company,4570.000,32136.000,17136.000,7070.000,44659.000,34533.000,10063.000
8,281586,91142,2016-12-31,AOS,A.O. Smith Corp,326.500,2685.900,1566.600,446.600,2891.000,1375.700,1515.300
9,281586,91142,2017-12-31,AOS,A.O. Smith Corp,296.500,2996.700,1758.000,326.400,3197.300,1548.500,1648.800


Let's restrict the sample to a set of large tech firms in the USA.

In [3]:
tech = ['AAPL','GOOGL','MSFT', 'IBM','INTC','CSCO','FB', 'ADBE', 'ORCL','CRM']
comp_tech =  comp1[comp1['tic'].isin(tech)]
pd.DataFrame(comp_tech.conm.unique())

,0
0,Adobe Inc.
1,Alphabet Inc. (Class A)
2,Apple Inc.
3,Cisco Systems
4,"Facebook, Inc."
5,Intel Corp.
6,International Business Machines
7,Microsoft Corp.
8,Oracle Corp.
9,Salesforce.com


In [4]:
comp_tech.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80 entries, 90 to 2830
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   gvkey     80 non-null     object        
 1   cik       80 non-null     int64         
 2   datadate  80 non-null     datetime64[ns]
 3   tic       80 non-null     object        
 4   conm      80 non-null     object        
 5   ni        80 non-null     float64       
 6   sale      80 non-null     float64       
 7   cogs      70 non-null     float64       
 8   oancf     67 non-null     float64       
 9   at        79 non-null     float64       
 10  lt        79 non-null     float64       
 11  seq       80 non-null     float64       
dtypes: datetime64[ns](1), float64(7), int64(1), object(3)
memory usage: 8.1+ KB


#### One-to-one merge

Let's merge the tech company financial statement data (`comp_tech`) with a linktable (`linktable`) that includes a host of identifying variables.

In [5]:
#linktable = pd.read_excel('linktable.xlsx',index=False,)
linktable = pd.read_excel('linktable.xlsx')
linktable.head()

,tic,cik,permno,gvkey
0,QRVO,1604778,3582,481822
1,ICE,1571949,60498,33871
2,PRGO,1585364,53227,613134
3,MDT,1613103,21333,473521
4,FTI,1681459,3885,720180


We need to clean up the identifiers to facilitate the merge.  They should be as follows:
 - CIK = 10 digit identifier from the SEC
 - GVKEY = 6-digit identifier from Compustat
 - PERMNO = 5-digit identifier from CRSP

In [6]:
linktable['cik'] = linktable['cik'].apply('{:0>10}'.format)
linktable['gvkey'] = linktable['gvkey'].apply('{:0>6}'.format)
linktable['permno'] = linktable['permno'].apply('{:0>5}'.format)

Let's merge the two, with `comp_tech` on the left and `linktable` on the right.  The common identifier is `gvkey`.

In [7]:
comp_tech2 = pd.merge(comp_tech, linktable[['gvkey','permno']], how='inner', on=['gvkey'])
comp_tech2.head(10)

,gvkey,cik,datadate,tic,conm,ni,sale,cogs,oancf,at,lt,seq,permno
0,597882,796343,2012-11-30,ADBE,Adobe Inc.,832.775,4403.677,121.663,1499.580,9974.523,3309.341,6665.182,04368
1,597882,796343,2013-11-30,ADBE,Adobe Inc.,289.985,4055.240,138.154,1151.686,10380.298,3655.664,6724.634,04368
2,597882,796343,2014-11-30,ADBE,Adobe Inc.,268.395,4147.065,622.080,1287.482,10785.829,4009.924,6775.905,04368
3,597882,796343,2015-11-30,ADBE,Adobe Inc.,629.551,4795.511,90.035,1469.502,11726.472,4724.892,7001.580,04368
4,597882,796343,2016-11-30,ADBE,Adobe Inc.,1168.782,5854.430,68.917,2199.728,12707.114,5282.279,7424.835,04368
5,597882,796343,2017-11-30,ADBE,Adobe Inc.,1693.954,7301.505,57.082,2912.853,14535.556,6075.687,8459.869,04368
6,597882,796343,2018-11-30,ADBE,Adobe Inc.,2590.774,9030.008,1194.999,4029.304,18768.682,9406.568,9362.114,04368
7,597882,796343,2019-11-30,ADBE,Adobe Inc.,2951.458,11171.297,1672.720,4421.813,20762.400,10232.245,10530.155,04368
8,765942,1652044,2013-12-31,GOOGL,Alphabet Inc. (Class A),12733.000,55519.000,21993.000,18659.000,NaN,NaN,86977.000,69132
9,765942,1652044,2014-12-31,GOOGL,Alphabet Inc. (Class A),14136.000,66001.000,25691.000,22376.000,130426.000,26566.000,103860.000,69132


In [8]:
comp_tech2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   gvkey     80 non-null     object        
 1   cik       80 non-null     int64         
 2   datadate  80 non-null     datetime64[ns]
 3   tic       80 non-null     object        
 4   conm      80 non-null     object        
 5   ni        80 non-null     float64       
 6   sale      80 non-null     float64       
 7   cogs      70 non-null     float64       
 8   oancf     67 non-null     float64       
 9   at        79 non-null     float64       
 10  lt        79 non-null     float64       
 11  seq       80 non-null     float64       
 12  permno    80 non-null     object        
dtypes: datetime64[ns](1), float64(7), int64(1), object(4)
memory usage: 8.3+ KB


### Another one-to-one merge

Let's bring in the year-end stock price for each firm.  We'll do this by merging `comp_tech` with the simulated CRSP daily file, `crspd`.

The common identifiers between the two datasets are: **permno** and **date**.

First, let's read in simulated CRSP daily stock return data. As before, make sure the permno has exactly 5 digits.

In [9]:
#crspd = pd.read_excel('dsf.xlsx', index=False, parse_dates=['date'], converters={'permno': '{:0>5}'.format})
crspd = pd.read_excel('dsf.xlsx', parse_dates=['date'], converters={'permno': '{:0>5}'.format})
crspd.head()

,permno,date,vol,prc,ret
0,92040,2012-01-03,3380100.0,66.56,0.113376
1,92040,2012-01-04,3007400.0,67.11,0.008263
2,92040,2012-01-05,3116400.0,66.81,-0.004470
3,92040,2012-01-06,2839200.0,66.46,-0.005239
4,92040,2012-01-09,2796600.0,66.86,0.006019


In [10]:
crspd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972261 entries, 0 to 972260
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   permno  972261 non-null  object        
 1   date    972261 non-null  datetime64[ns]
 2   vol     969679 non-null  float64       
 3   prc     972261 non-null  float64       
 4   ret     972261 non-null  float64       
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 37.1+ MB


What do you notice?  There are only about 80 observations in `comp_tech`, but there are nearly a million in `crspd`.  We'd better merge carefully, or we'll end up with a giant (or bad) dataset. 

In [11]:
m1 = pd.merge(comp_tech2, crspd[['permno', 'date', 'prc']], how='left', left_on=['permno', 'datadate'], right_on=['permno', 'date'])

m1.head()

,gvkey,cik,datadate,tic,conm,ni,sale,cogs,oancf,at,lt,seq,permno,date,prc
0,597882,796343,2012-11-30,ADBE,Adobe Inc.,832.775,4403.677,121.663,1499.580,9974.523,3309.341,6665.182,04368,2012-11-30,34.61
1,597882,796343,2013-11-30,ADBE,Adobe Inc.,289.985,4055.240,138.154,1151.686,10380.298,3655.664,6724.634,04368,NaT,NaN
2,597882,796343,2014-11-30,ADBE,Adobe Inc.,268.395,4147.065,622.080,1287.482,10785.829,4009.924,6775.905,04368,NaT,NaN
3,597882,796343,2015-11-30,ADBE,Adobe Inc.,629.551,4795.511,90.035,1469.502,11726.472,4724.892,7001.580,04368,2015-11-30,91.46
4,597882,796343,2016-11-30,ADBE,Adobe Inc.,1168.782,5854.430,68.917,2199.728,12707.114,5282.279,7424.835,04368,2016-11-30,102.81


In [12]:
m1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   gvkey     80 non-null     object        
 1   cik       80 non-null     int64         
 2   datadate  80 non-null     datetime64[ns]
 3   tic       80 non-null     object        
 4   conm      80 non-null     object        
 5   ni        80 non-null     float64       
 6   sale      80 non-null     float64       
 7   cogs      70 non-null     float64       
 8   oancf     67 non-null     float64       
 9   at        79 non-null     float64       
 10  lt        79 non-null     float64       
 11  seq       80 non-null     float64       
 12  permno    80 non-null     object        
 13  date      56 non-null     datetime64[ns]
 14  prc       56 non-null     float64       
dtypes: datetime64[ns](2), float64(8), int64(1), object(4)
memory usage: 9.5+ KB


###  Why are we losing data?

It's probably because our merge-on variables (permno, date) don't align for some reason.  

When this happens, you want to double check the identifiers in each dataset.  That is, we need to make sure that Python views `permno` and `date` equivalently in each dataset.  If not, the observations won't merge properly.

We know that the `permno` is measured correctly, because we formatted it in both datasets up above.  Thus, it's very likely that it's an issue with the dates.

So, the first thing we need to do is change up the dates.  

In [13]:
# create a year variable using pandas datetimeindex
crspd['year'] = pd.DatetimeIndex(crspd['date']).year
crspd['month'] = pd.DatetimeIndex(crspd['date']).month
crspd.head()


,permno,date,vol,prc,ret,year,month
0,92040,2012-01-03,3380100.0,66.56,0.113376,2012,1
1,92040,2012-01-04,3007400.0,67.11,0.008263,2012,1
2,92040,2012-01-05,3116400.0,66.81,-0.004470,2012,1
3,92040,2012-01-06,2839200.0,66.46,-0.005239,2012,1
4,92040,2012-01-09,2796600.0,66.86,0.006019,2012,1


In [14]:
#sort values by permno and date, then -- within each permo-year-month grouping -- take the last observation
crsp2 = crspd.sort_values(['permno','year','month']).groupby(['permno','year','month']).tail(1)
crsp2.head()

,permno,date,vol,prc,ret,year,month
87352,00184,2012-01-31,2617500.0,84.66,0.000236,2012,1
87372,00184,2012-02-29,2829300.0,92.41,-0.015134,2012,2
87394,00184,2012-03-30,2388600.0,86.00,0.016548,2012,3
87414,00184,2012-04-30,2185000.0,82.29,0.010809,2012,4
87436,00184,2012-05-31,3532700.0,69.81,-0.001859,2012,5


In [15]:
crsp2[(crsp2['permno']=='93175') & (crsp2['month'] == 9)]

,permno,date,vol,prc,ret,year,month
91503,93175,2012-09-28,133777700.0,82.86,-0.020799,2012,9
91753,93175,2013-09-30,65039100.0,60.62,-0.012543,2013,9
92005,93175,2014-09-30,55264100.0,91.70,0.006365,2014,9
92257,93175,2015-09-30,66473000.0,102.11,0.011290,2015,9
92510,93175,2016-09-30,36379100.0,106.90,0.007825,2016,9
92761,93175,2017-09-29,26299800.0,148.30,0.005492,2017,9
93012,93175,2018-09-28,22929400.0,220.50,0.003504,2018,9
93263,93175,2019-09-30,25977400.0,222.17,0.023542,2019,9


In [16]:
comp_tech2[comp_tech2['permno']=='93175']

,gvkey,cik,datadate,tic,conm,ni,sale,cogs,oancf,at,lt,seq,permno
15,107764,320193,2012-09-30,AAPL,Apple Inc.,41733.0,156508.0,87846.0,50856.0,176064.0,57854.0,118210.0,93175
16,107764,320193,2013-09-30,AAPL,Apple Inc.,37037.0,170910.0,106606.0,53666.0,207000.0,83451.0,123549.0,93175
17,107764,320193,2014-09-30,AAPL,Apple Inc.,39510.0,182795.0,112258.0,NaN,231839.0,120292.0,111547.0,93175
18,107764,320193,2015-09-30,AAPL,Apple Inc.,53394.0,233715.0,140089.0,81266.0,290479.0,171124.0,119355.0,93175
19,107764,320193,2016-09-30,AAPL,Apple Inc.,45687.0,215639.0,131376.0,65824.0,321686.0,193437.0,128249.0,93175
20,107764,320193,2017-09-30,AAPL,Apple Inc.,48351.0,229234.0,141048.0,63598.0,375319.0,241272.0,134047.0,93175
21,107764,320193,2018-09-30,AAPL,Apple Inc.,59531.0,265595.0,163756.0,77434.0,365725.0,258578.0,107147.0,93175
22,107764,320193,2019-09-30,AAPL,Apple Inc.,55256.0,260174.0,161782.0,69391.0,338516.0,248028.0,90488.0,93175


In [17]:
crsp2.drop(['date','vol','ret'], axis=1, inplace=True)
crsp2.head()

,permno,prc,year,month
87352,00184,84.66,2012,1
87372,00184,92.41,2012,2
87394,00184,86.00,2012,3
87414,00184,82.29,2012,4
87436,00184,69.81,2012,5


In [18]:
comp_tech2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   gvkey     80 non-null     object        
 1   cik       80 non-null     int64         
 2   datadate  80 non-null     datetime64[ns]
 3   tic       80 non-null     object        
 4   conm      80 non-null     object        
 5   ni        80 non-null     float64       
 6   sale      80 non-null     float64       
 7   cogs      70 non-null     float64       
 8   oancf     67 non-null     float64       
 9   at        79 non-null     float64       
 10  lt        79 non-null     float64       
 11  seq       80 non-null     float64       
 12  permno    80 non-null     object        
dtypes: datetime64[ns](1), float64(7), int64(1), object(4)
memory usage: 8.3+ KB


In [19]:
comp_tech2['year'] = pd.DatetimeIndex(comp_tech2['datadate']).year
comp_tech2['month'] = pd.DatetimeIndex(comp_tech2['datadate']).month

In [20]:
comp_tech2.head()

,gvkey,cik,datadate,tic,conm,ni,sale,cogs,oancf,at,lt,seq,permno,year,month
0,597882,796343,2012-11-30,ADBE,Adobe Inc.,832.775,4403.677,121.663,1499.580,9974.523,3309.341,6665.182,04368,2012,11
1,597882,796343,2013-11-30,ADBE,Adobe Inc.,289.985,4055.240,138.154,1151.686,10380.298,3655.664,6724.634,04368,2013,11
2,597882,796343,2014-11-30,ADBE,Adobe Inc.,268.395,4147.065,622.080,1287.482,10785.829,4009.924,6775.905,04368,2014,11
3,597882,796343,2015-11-30,ADBE,Adobe Inc.,629.551,4795.511,90.035,1469.502,11726.472,4724.892,7001.580,04368,2015,11
4,597882,796343,2016-11-30,ADBE,Adobe Inc.,1168.782,5854.430,68.917,2199.728,12707.114,5282.279,7424.835,04368,2016,11


In [21]:
m1 = pd.merge(comp_tech2, crsp2, how='left', on=['permno','month','year'])

m1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   gvkey     80 non-null     object        
 1   cik       80 non-null     int64         
 2   datadate  80 non-null     datetime64[ns]
 3   tic       80 non-null     object        
 4   conm      80 non-null     object        
 5   ni        80 non-null     float64       
 6   sale      80 non-null     float64       
 7   cogs      70 non-null     float64       
 8   oancf     67 non-null     float64       
 9   at        79 non-null     float64       
 10  lt        79 non-null     float64       
 11  seq       80 non-null     float64       
 12  permno    80 non-null     object        
 13  year      80 non-null     int32         
 14  month     80 non-null     int32         
 15  prc       79 non-null     float64       
dtypes: datetime64[ns](1), float64(8), int32(2), int64(1), object(4)


In [22]:
m1[m1.prc.isnull()]

,gvkey,cik,datadate,tic,conm,ni,sale,cogs,oancf,at,lt,seq,permno,year,month,prc
79,821276,1108524,2020-01-31,CRM,Salesforce.com,126.0,17098.0,4235.0,4331.0,55126.0,21241.0,33885.0,44205,2020,1,NaN


In [23]:
m1.rename(columns={'prc':'end_price'}, inplace=True)


In [24]:
m1.tail(10)

,gvkey,cik,datadate,tic,conm,ni,sale,cogs,oancf,at,lt,seq,permno,year,month,end_price
70,215379,1341439,2019-05-31,ORCL,Oracle Corp.,11083.000,39506.000,NaN,14551.000,108709.000,86924.000,21785.000,98315,2019,5,49.71
71,821276,1108524,2012-01-31,CRM,Salesforce.com,-11.572,2266.539,488.886,591.507,4164.154,2498.053,1587.360,44205,2012,1,29.20
72,821276,1108524,2013-01-31,CRM,Salesforce.com,-270.445,3050.195,683.579,736.897,5528.956,3157.711,2317.633,44205,2013,1,43.03
73,821276,1108524,2014-01-31,CRM,Salesforce.com,-232.175,4071.003,968.428,875.469,9152.930,6087.715,3038.510,44205,2014,1,60.53
74,821276,1108524,2015-01-31,CRM,Salesforce.com,-262.688,5373.586,NaN,1173.714,10692.982,6717.799,3975.183,44205,2015,1,56.45
75,821276,1108524,2016-01-31,CRM,Salesforce.com,-47.426,6667.216,NaN,1612.585,12770.772,7767.903,5002.869,44205,2016,1,68.06
76,821276,1108524,2017-01-31,CRM,Salesforce.com,179.632,8391.984,2234.000,2162.198,17584.923,10084.796,7500.127,44205,2017,1,79.10
77,821276,1108524,2018-01-31,CRM,Salesforce.com,127.478,10540.000,2773.000,2737.965,21009.802,11617.439,9388.496,44205,2018,1,113.91
78,821276,1108524,2019-01-31,CRM,Salesforce.com,1110.000,13282.000,3451.000,3398.000,30737.000,15132.000,15605.000,44205,2019,1,151.97
79,821276,1108524,2020-01-31,CRM,Salesforce.com,126.000,17098.000,4235.000,4331.000,55126.000,21241.000,33885.000,44205,2020,1,NaN


### One to many merge, relative dates, groupby 



In [25]:
appl = comp_tech2[comp_tech2.tic == 'AAPL']
appl

,gvkey,cik,datadate,tic,conm,ni,sale,cogs,oancf,at,lt,seq,permno,year,month
15,107764,320193,2012-09-30,AAPL,Apple Inc.,41733.0,156508.0,87846.0,50856.0,176064.0,57854.0,118210.0,93175,2012,9
16,107764,320193,2013-09-30,AAPL,Apple Inc.,37037.0,170910.0,106606.0,53666.0,207000.0,83451.0,123549.0,93175,2013,9
17,107764,320193,2014-09-30,AAPL,Apple Inc.,39510.0,182795.0,112258.0,NaN,231839.0,120292.0,111547.0,93175,2014,9
18,107764,320193,2015-09-30,AAPL,Apple Inc.,53394.0,233715.0,140089.0,81266.0,290479.0,171124.0,119355.0,93175,2015,9
19,107764,320193,2016-09-30,AAPL,Apple Inc.,45687.0,215639.0,131376.0,65824.0,321686.0,193437.0,128249.0,93175,2016,9
20,107764,320193,2017-09-30,AAPL,Apple Inc.,48351.0,229234.0,141048.0,63598.0,375319.0,241272.0,134047.0,93175,2017,9
21,107764,320193,2018-09-30,AAPL,Apple Inc.,59531.0,265595.0,163756.0,77434.0,365725.0,258578.0,107147.0,93175,2018,9
22,107764,320193,2019-09-30,AAPL,Apple Inc.,55256.0,260174.0,161782.0,69391.0,338516.0,248028.0,90488.0,93175,2019,9


Merge in stock prices for all days in the fiscal year;


In [ ]:
from pandas.tseries.offsets import MonthBegin
from datetime import timedelta

# Wrong way of getting the first date of fiscal year
#comp_tech2['begdate'] = comp_tech2['datadate']- timedelta(days=365)

comp_tech2['begdate'] = comp_tech2['datadate'] - timedelta(days=365) + MonthBegin(0)

comp_tech2[['tic','begdate','datadate']].head(11)

,tic,begdate,datadate
0,ADBE,2011-12-01,2012-11-30
1,ADBE,2012-12-01,2013-11-30
2,ADBE,2013-12-01,2014-11-30
3,ADBE,2014-12-01,2015-11-30
4,ADBE,2015-12-01,2016-11-30
5,ADBE,2016-12-01,2017-11-30
6,ADBE,2017-12-01,2018-11-30
7,ADBE,2018-12-01,2019-11-30
8,GOOGL,2013-01-01,2013-12-31
9,GOOGL,2014-01-01,2014-12-31


In [27]:
comp_tech2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   gvkey     80 non-null     object        
 1   cik       80 non-null     int64         
 2   datadate  80 non-null     datetime64[ns]
 3   tic       80 non-null     object        
 4   conm      80 non-null     object        
 5   ni        80 non-null     float64       
 6   sale      80 non-null     float64       
 7   cogs      70 non-null     float64       
 8   oancf     67 non-null     float64       
 9   at        79 non-null     float64       
 10  lt        79 non-null     float64       
 11  seq       80 non-null     float64       
 12  permno    80 non-null     object        
 13  year      80 non-null     int32         
 14  month     80 non-null     int32         
 15  begdate   80 non-null     datetime64[ns]
dtypes: datetime64[ns](2), float64(7), int32(2), int64(1), object(4)


In [ ]:
import sqlite3
sqlconn = sqlite3.connect(':memory:')
comp_tech2.to_sql('comp_tech2',sqlconn, index=False)
crspd.to_sql('crspd', sqlconn, index=False)

qry = '''
     SELECT
    a.tic, a.permno, a.begdate, a.datadate, a.year, b.date, b.prc
    FROM comp_tech2 as a 
    LEFT JOIN crspd as b
    ON a.permno = b.permno
    AND a.begdate <= b.date 
    AND b.date <= a.datadate
    ORDER BY gvkey, datadate, date
    '''

In [ ]:
temp2 = pd.read_sql_query(qry, sqlconn)

In [30]:
# we should have roughly 20,000 observations = 80 firm-years * about 250 trading days
temp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19308 entries, 0 to 19307
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tic       19308 non-null  object 
 1   permno    19308 non-null  object 
 2   begdate   19308 non-null  object 
 3   datadate  19308 non-null  object 
 4   year      19308 non-null  int64  
 5   date      19308 non-null  object 
 6   prc       19308 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 1.0+ MB


In [31]:
temp2[(temp2['tic']=='MSFT') & (temp2['year']==2016)]

,tic,permno,begdate,datadate,year,date,prc
878,MSFT,05616,2015-07-01 00:00:00,2016-06-30 00:00:00,2016,2015-07-01 00:00:00,40.23
879,MSFT,05616,2015-07-01 00:00:00,2016-06-30 00:00:00,2016,2015-07-02 00:00:00,40.18
880,MSFT,05616,2015-07-01 00:00:00,2016-06-30 00:00:00,2016,2015-07-06 00:00:00,40.17
881,MSFT,05616,2015-07-01 00:00:00,2016-06-30 00:00:00,2016,2015-07-07 00:00:00,40.09
882,MSFT,05616,2015-07-01 00:00:00,2016-06-30 00:00:00,2016,2015-07-08 00:00:00,40.04
...,...,...,...,...,...,...,...
1126,MSFT,05616,2015-07-01 00:00:00,2016-06-30 00:00:00,2016,2016-06-24 00:00:00,46.35
1127,MSFT,05616,2015-07-01 00:00:00,2016-06-30 00:00:00,2016,2016-06-27 00:00:00,45.05
1128,MSFT,05616,2015-07-01 00:00:00,2016-06-30 00:00:00,2016,2016-06-28 00:00:00,45.99
1129,MSFT,05616,2015-07-01 00:00:00,2016-06-30 00:00:00,2016,2016-06-29 00:00:00,47.01


In [32]:
temp3 = temp2.groupby(['permno','year'], as_index=False)['prc'].mean()
temp3.rename(columns={'prc':'avg_prc'}, inplace=True)
temp3.head(10)

,permno,year,avg_prc
0,04368,2012,32.328783
1,04368,2013,45.084104
2,04368,2014,66.121355
3,04368,2015,79.269802
4,04368,2016,96.381028
5,04368,2017,138.233730
6,04368,2018,230.471627
7,04368,2019,272.528440
8,05616,2012,25.196960
9,05616,2013,25.079438


In [33]:
comp_tech3=pd.merge(comp_tech2, temp3, how='left', on=['permno','year'])
comp_tech3.head()

,gvkey,cik,datadate,tic,conm,ni,sale,cogs,oancf,at,lt,seq,permno,year,month,begdate,avg_prc
0,597882,796343,2012-11-30,ADBE,Adobe Inc.,832.775,4403.677,121.663,1499.580,9974.523,3309.341,6665.182,04368,2012,11,2011-12-01,32.328783
1,597882,796343,2013-11-30,ADBE,Adobe Inc.,289.985,4055.240,138.154,1151.686,10380.298,3655.664,6724.634,04368,2013,11,2012-12-01,45.084104
2,597882,796343,2014-11-30,ADBE,Adobe Inc.,268.395,4147.065,622.080,1287.482,10785.829,4009.924,6775.905,04368,2014,11,2013-12-01,66.121355
3,597882,796343,2015-11-30,ADBE,Adobe Inc.,629.551,4795.511,90.035,1469.502,11726.472,4724.892,7001.580,04368,2015,11,2014-12-01,79.269802
4,597882,796343,2016-11-30,ADBE,Adobe Inc.,1168.782,5854.430,68.917,2199.728,12707.114,5282.279,7424.835,04368,2016,11,2015-12-01,96.381028


In [34]:
comp_tech3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   gvkey     80 non-null     object        
 1   cik       80 non-null     int64         
 2   datadate  80 non-null     datetime64[ns]
 3   tic       80 non-null     object        
 4   conm      80 non-null     object        
 5   ni        80 non-null     float64       
 6   sale      80 non-null     float64       
 7   cogs      70 non-null     float64       
 8   oancf     67 non-null     float64       
 9   at        79 non-null     float64       
 10  lt        79 non-null     float64       
 11  seq       80 non-null     float64       
 12  permno    80 non-null     object        
 13  year      80 non-null     int32         
 14  month     80 non-null     int32         
 15  begdate   80 non-null     datetime64[ns]
 16  avg_prc   80 non-null     float64       
dtypes: datetime64[ns](

### Compute stock returns

Let's first bring in a scraped dataset of monthly stock returns: `msf`

In [35]:
msf = pd.read_excel('msf.xlsx', parse_dates=['date'])
msf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46411 entries, 0 to 46410
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   permno  46411 non-null  int64         
 1   ret     46411 non-null  float64       
 2   date    46411 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 1.1 MB


In [36]:
msf.head()

,permno,ret,date
0,184,0.149353,2012-01-31
1,184,0.091543,2012-02-29
2,184,-0.069365,2012-03-31
3,184,-0.043140,2012-04-30
4,184,-0.151659,2012-05-31


In [37]:
msf['permno'] = msf['permno'].apply('{:0>5}'.format)
msf['year']=msf['date'].dt.year
msf.head()

,permno,ret,date,year
0,00184,0.149353,2012-01-31,2012
1,00184,0.091543,2012-02-29,2012
2,00184,-0.069365,2012-03-31,2012
3,00184,-0.043140,2012-04-30,2012
4,00184,-0.151659,2012-05-31,2012


##### Merge together with the S&P500 return

In [38]:
#bring in the monthly S&P 500 return
mkt = pd.read_excel('msi_sp500.xlsx', parse_dates=['Date'])
mkt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   Date    96 non-null     datetime64[ns]
 1   EWRETD  96 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.6 KB


In [39]:
#convert all column names to lower case
mkt.columns = map(str.lower, mkt.columns)
mkt.head()

,date,ewretd
0,2012-01-31,0.053011
1,2012-02-29,0.043405
2,2012-03-30,0.027660
3,2012-04-30,-0.002322
4,2012-05-31,-0.060056


In [40]:
#make sure the dates match by converting to MonthEnd
from pandas.tseries.offsets import MonthEnd
msf['date']= msf['date']+MonthEnd(0)
mkt['date']= mkt['date']+MonthEnd(0)

In [41]:
msf = pd.merge(msf, mkt, on=['date'])

msf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46411 entries, 0 to 46410
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   permno  46411 non-null  object        
 1   ret     46411 non-null  float64       
 2   date    46411 non-null  datetime64[ns]
 3   year    46411 non-null  int32         
 4   ewretd  46411 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int32(1), object(1)
memory usage: 1.6+ MB


In [42]:
msf.sort_values(['permno','date'], inplace=True)
msf.head(12)

,permno,ret,date,year,ewretd
0,00184,0.149353,2012-01-31,2012,0.053011
462,00184,0.091543,2012-02-29,2012,0.043405
925,00184,-0.069365,2012-03-31,2012,0.027660
1387,00184,-0.043140,2012-04-30,2012,-0.002322
1850,00184,-0.151659,2012-05-31,2012,-0.060056
2315,00184,0.079931,2012-06-30,2012,0.035217
2782,00184,-0.018172,2012-07-31,2012,0.017072
3248,00184,-0.004323,2012-08-31,2012,0.025053
3714,00184,0.008412,2012-09-30,2012,0.019906
4179,00184,-0.041577,2012-10-31,2012,-0.012957


###### Compute cumulative returns for both the firm and the market

We are going to compute *annual* cumulative returns from monthly returns.  To so do, we are going to:
1. compute the monthly logged return
2. sum over log(1 + ret) for the year 
3. then take the anti-log - 1.



In [43]:
msf['logret']=np.log(1+msf['ret'])
msf['logmktret']=np.log(1+msf['ewretd'])
msf.describe()

,ret,date,year,ewretd,logret,logmktret
count,46411.000000,46411,46411.000000,46411.000000,46411.000000,46411.000000
mean,0.014572,2016-01-30 10:54:23.588373504,2015.541790,0.012042,0.011184,0.011451
min,-0.462013,2012-01-31 00:00:00,2012.000000,-0.093343,-0.619920,-0.097991
25%,-0.026846,2014-01-31 00:00:00,2014.000000,-0.003087,-0.027213,-0.003092
50%,0.014711,2016-01-31 00:00:00,2016.000000,0.014772,0.014604,0.014664
75%,0.054263,2018-01-31 00:00:00,2018.000000,0.029638,0.052842,0.029207
max,9.620000,2019-12-31 00:00:00,2019.000000,0.095736,2.362739,0.091426
std,0.119445,NaN,2.284654,0.032392,0.075044,0.032311


In [ ]:
temp4 = msf.groupby(['permno','year'], as_index=False)[['logret', 'logmktret']].sum()
temp4.head()

,permno,year,logret,logmktret
0,00184,2012,-0.085836,0.147591
1,00184,2013,0.100140,0.281614
2,00184,2014,-0.304598,0.126224
3,00184,2015,-0.341128,0.011838
4,00184,2016,0.376152,0.096498


In [45]:
temp4['annret']=np.exp(temp4['logret'])-1
temp4['annmktret']=np.exp(temp4['logmktret'])-1
temp4.drop(columns=['logret', 'logmktret'], axis=1, inplace=True)
temp4.head()

,permno,year,annret,annmktret
0,00184,2012,-0.082255,0.159039
1,00184,2013,0.105325,0.325267
2,00184,2014,-0.262580,0.134536
3,00184,2015,-0.289032,0.011908
4,00184,2016,0.456669,0.101308


In [46]:
temp4['ann_abn_ret'] = temp4['annret']-temp4['annmktret']

##### Merge annual returns with `comp_tech2`

In [47]:
comp_tech2.head()

,gvkey,cik,datadate,tic,conm,ni,sale,cogs,oancf,at,lt,seq,permno,year,month,begdate
0,597882,796343,2012-11-30,ADBE,Adobe Inc.,832.775,4403.677,121.663,1499.580,9974.523,3309.341,6665.182,04368,2012,11,2011-12-01
1,597882,796343,2013-11-30,ADBE,Adobe Inc.,289.985,4055.240,138.154,1151.686,10380.298,3655.664,6724.634,04368,2013,11,2012-12-01
2,597882,796343,2014-11-30,ADBE,Adobe Inc.,268.395,4147.065,622.080,1287.482,10785.829,4009.924,6775.905,04368,2014,11,2013-12-01
3,597882,796343,2015-11-30,ADBE,Adobe Inc.,629.551,4795.511,90.035,1469.502,11726.472,4724.892,7001.580,04368,2015,11,2014-12-01
4,597882,796343,2016-11-30,ADBE,Adobe Inc.,1168.782,5854.430,68.917,2199.728,12707.114,5282.279,7424.835,04368,2016,11,2015-12-01


In [48]:
comp_tech3 = pd.merge(comp_tech2, temp4, on=['permno','year'])
comp_tech3.head()

,gvkey,cik,datadate,tic,conm,ni,sale,cogs,oancf,at,lt,seq,permno,year,month,begdate,annret,annmktret,ann_abn_ret
0,597882,796343,2012-11-30,ADBE,Adobe Inc.,832.775,4403.677,121.663,1499.580,9974.523,3309.341,6665.182,04368,2012,11,2011-12-01,0.159776,0.159039,0.000737
1,597882,796343,2013-11-30,ADBE,Adobe Inc.,289.985,4055.240,138.154,1151.686,10380.298,3655.664,6724.634,04368,2013,11,2012-12-01,0.568846,0.325267,0.243579
2,597882,796343,2014-11-30,ADBE,Adobe Inc.,268.395,4147.065,622.080,1287.482,10785.829,4009.924,6775.905,04368,2014,11,2013-12-01,0.214095,0.134536,0.079559
3,597882,796343,2015-11-30,ADBE,Adobe Inc.,629.551,4795.511,90.035,1469.502,11726.472,4724.892,7001.580,04368,2015,11,2014-12-01,0.292160,0.011908,0.280252
4,597882,796343,2016-11-30,ADBE,Adobe Inc.,1168.782,5854.430,68.917,2199.728,12707.114,5282.279,7424.835,04368,2016,11,2015-12-01,0.095912,0.120043,-0.024131


In [49]:
comp_tech3.to_csv('comp_tech3.csv', index=False)

### Ranking variables into groups

In [50]:
comp_tech3 = pd.read_csv('comp_tech3.csv', index_col=False)
comp_tech3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 19 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   gvkey        79 non-null     int64  
 1   cik          79 non-null     int64  
 2   datadate     79 non-null     object 
 3   tic          79 non-null     object 
 4   conm         79 non-null     object 
 5   ni           79 non-null     float64
 6   sale         79 non-null     float64
 7   cogs         69 non-null     float64
 8   oancf        66 non-null     float64
 9   at           78 non-null     float64
 10  lt           78 non-null     float64
 11  seq          79 non-null     float64
 12  permno       79 non-null     int64  
 13  year         79 non-null     int64  
 14  month        79 non-null     int64  
 15  begdate      79 non-null     object 
 16  annret       79 non-null     float64
 17  annmktret    79 non-null     float64
 18  ann_abn_ret  79 non-null     float64
dtypes: float64

In [ ]:
comp_tech3['roe'] = comp_tech3['ni']/comp_tech3['seq']
comp_tech3['roe'].describe()

count    79.000000
mean      0.237551
std       0.212858
min      -0.116690
25%       0.135982
50%       0.186953
75%       0.287625
max       1.012976
Name: roe, dtype: float64

In [ ]:
add_10 = lambda x: x + 10
print(add_10(5)) 

15


In [53]:
#rank ROE into deciles
comp_tech3['r10_roe']=comp_tech3['roe'].transform(lambda x: pd.qcut(x, 10, labels=False))

#rank ROE into quintiles
comp_tech3['r5_roe']=comp_tech3['roe'].transform(lambda x: pd.qcut(x, 5, labels=False))+1  #add 1 to change the range to [1,5]

#rank ROE into an indicator
comp_tech3['high_roe']=comp_tech3['roe'].transform(lambda x: pd.qcut(x, 2, labels=False))

comp_tech3[['tic','year','roe','r10_roe', 'r5_roe', 'high_roe']].head(20)

,tic,year,roe,r10_roe,r5_roe,high_roe
0,ADBE,2012,0.124944,2,2,0
1,ADBE,2013,0.043123,1,1,0
2,ADBE,2014,0.039610,1,1,0
3,ADBE,2015,0.089916,2,2,0
4,ADBE,2016,0.157415,3,2,0
5,ADBE,2017,0.200234,5,3,1
6,ADBE,2018,0.276730,7,4,1
7,ADBE,2019,0.280286,7,4,1
8,GOOGL,2013,0.146395,3,2,0
9,GOOGL,2014,0.136106,2,2,0


In [54]:
comp_tech3['r10_roe'].value_counts()

r10_roe
2    8
1    8
3    8
7    8
4    8
8    8
9    8
0    8
6    8
5    7
Name: count, dtype: int64

It's often more appropriate to do the ranks by some group (e.g., year, industry).  Here's how to do that:

In [55]:
#rank ROE into quintiles, by year
comp_tech3['yr_r5_roe']=comp_tech3.groupby(['year'])['roe'].transform(lambda x: pd.qcut(x, 5, labels=False, duplicates='drop'))

In [56]:
comp_tech3['yr_r5_roe'].describe()

count    79.00000
mean      2.00000
std       1.43223
min       0.00000
25%       1.00000
50%       2.00000
75%       3.00000
max       4.00000
Name: yr_r5_roe, dtype: float64

### Summarizing data by groups

In [57]:
comp_tech3['pm'] = comp_tech3['ni']/comp_tech3['sale']
comp_tech3['turn'] = comp_tech3['sale']/comp_tech3['at']
comp_tech3['lev'] = comp_tech3['at']/comp_tech3['seq']

Let's start with a single aggregation: mean()

In [58]:
comp_tech3.groupby('r5_roe', as_index=False)[['pm','turn','lev']].mean()

,r5_roe,pm,turn,lev
0,1,0.044434,0.441693,2.080021
1,2,0.184461,0.479529,1.634465
2,3,0.242353,0.477075,1.789660
3,4,0.256058,0.530325,2.340567
4,5,0.197401,0.693061,4.803156


Now, let's do multiple aggregations:

In [59]:
comp_tech3.groupby('r5_roe', as_index=False).agg(
            {"pm":['mean', 'median','std','var'], 
              "turn":['mean', 'median','std','var'],
            "lev":['mean', 'median','std','var']}) 

r5_roe        pm                                    turn            \
              mean    median       std       var      mean    median   
0      1  0.044434  0.016750  0.089144  0.007947  0.441693  0.449122   
1      2  0.184461  0.185897  0.029249  0.000855  0.479529  0.501978   
2      3  0.242353  0.224647  0.062543  0.003912  0.477075  0.480964   
3      4  0.256058  0.266539  0.067092  0.004501  0.530325  0.528939   
4      5  0.197401  0.211396  0.058651  0.003440  0.693061  0.732993   

                            lev                                
        std       var      mean    median       std       var  
0  0.087799  0.007709  2.080021  2.291220  0.665326  0.442658  
1  0.055366  0.003065  1.634465  1.711216  0.310929  0.096677  
2  0.107256  0.011504  1.789660  1.687239  0.566746  0.321201  
3  0.104060  0.010829  2.340567  1.876182  1.380670  1.906251  
4  0.150305  0.022592  4.803156  4.365545  2.494169  6.220879

#### Descriptive Statistics

In [60]:
varlist=['roe','pm','turn','lev']

In [61]:
comp_tech3[varlist].describe([.01,.25,.5,.75,.99])

,roe,pm,turn,lev
count,79.000000,79.000000,78.000000,78.000000
mean,0.237551,0.184215,0.525517,2.550536
std,0.212858,0.098358,0.137215,1.769530
min,-0.116690,-0.088665,0.279485,1.097425
1%,-0.085272,-0.063991,0.287719,1.109613
25%,0.135982,0.139915,0.439342,1.646376
50%,0.186953,0.206314,0.506484,1.916602
75%,0.287625,0.235786,0.583392,2.673281
99%,0.944226,0.392843,0.879467,8.243335
max,1.012976,0.396003,0.888927,9.903269


In [62]:
comp_tech3[varlist].corr(method='pearson')

,roe,pm,turn,lev
roe,1.000000,0.317454,0.642925,0.751316
pm,0.317454,1.000000,0.037075,-0.237123
turn,0.642925,0.037075,1.000000,0.398186
lev,0.751316,-0.237123,0.398186,1.000000


In [63]:
comp_tech3[varlist].corr(method='spearman')

,roe,pm,turn,lev
roe,1.000000,0.482400,0.532973,0.441748
pm,0.482400,1.000000,-0.012962,-0.293947
turn,0.532973,-0.012962,1.000000,0.177835
lev,0.441748,-0.293947,0.177835,1.000000


In [64]:
comp_tech3.to_csv('comp_tech4.csv', index=False)